In [63]:
import os
import pickle
import shutil

In [64]:
base_path = '../'
use_merged_dataset = False
generate_all_datset_annots = True
imgsz = 640
base_data_dir = 'dataset1_files' if not generate_all_datset_annots else 'all_dataset_files'
base_data_name = 'all_dataset1' if not generate_all_datset_annots else 'all_dataset'
train_modes = ['train', 'validation']
folds = [0]
# annot_dirs_to_convert = [f'{base_path}/dataset1_files/all_dataset1_annotations_merged_train_0', f'{base_path}/dataset1_files/all_dataset1_annotations_merged_validation_0']

In [65]:
for fold in folds:
  for train_mode in train_modes:
    yolo_output_dir = f'{base_path}/{base_data_dir}/yolo_dataset1_{train_mode}_{fold}'
    if os.path.exists(yolo_output_dir):
      shutil.rmtree(yolo_output_dir)
    os.mkdir(yolo_output_dir)
    os.mkdir(f'{yolo_output_dir}/images')
    os.mkdir(f'{yolo_output_dir}/labels')

In [66]:
def get_valid_annots(annots):
    valid_annots = []
    for annot in annots:
        if len(annot['segmentation'][0]) >= 6:
            valid_annots.append(annot)
    return valid_annots

In [67]:
for fold in folds:
  for train_mode in train_modes:
    yolo_output_dir = f'{base_path}/{base_data_dir}/yolo_dataset1_{train_mode}_{fold}'
    cur_img_dir = f'{base_path}/{base_data_dir}/{base_data_name}_imgs_merged_{train_mode}_{fold}' if use_merged_dataset else f'{base_path}/{base_data_dir}/{base_data_name}_{train_mode}_imgs_margins_{fold}'
    cur_annots_dir = f'{base_path}/{base_data_dir}/{base_data_name}_annotations_merged_{train_mode}_{fold}' if use_merged_dataset else f'{base_path}/{base_data_dir}/{base_data_name}_{train_mode}_annotations_margins_{fold}'
    for img in os.listdir(cur_img_dir):
      img_id = img.split('.png')[0]
      img_annot_file = f'{cur_annots_dir}/{img_id}.pkl'
      with open(img_annot_file, 'rb') as f:
        cur_annots = pickle.load(f)
      cur_annots = get_valid_annots(cur_annots)
      shutil.copy(f'{cur_img_dir}/{img}', f'{yolo_output_dir}/images/{img}')
      fp = open(f'{yolo_output_dir}/labels/{img_id}.txt', 'a')
      for cur_annot in cur_annots:
        top_left_x, top_left_y, width, height = cur_annot['bbox']
        if train_mode=='validation' or train_mode=='train' or (not (top_left_x < 2 or top_left_y < 2 or 512-(top_left_x+width) < 2 or  512-(top_left_y+height) < 2)) or (width > 10 and height > 10):
            cur_seg_relative = [float(coord)/imgsz for coord in cur_annot['segmentation'][0]]
            cur_seg_str = ' '.join([str(coord) for coord in cur_seg_relative])
            yolo_annot_line = f"{cur_annot['category_id']} {cur_seg_str}"
            fp.write(f'{yolo_annot_line}\n')
      fp.close()

In [26]:
train_mode = 'train'
fold = 0
for f in os.listdir(f'{base_path}/{base_data_dir}/yolo_dataset1_{train_mode}_{fold}/labels'):
    
    with open(f'{base_path}/{base_data_dir}/yolo_dataset1_{train_mode}_{fold}/labels/{f}', 'r') as f:
        s = f.read()
    if len(s.split()) < 200:
        print('=======')
        print(f)
        print(s)
        print('========')

<_io.TextIOWrapper name='..//all_dataset_files/yolo_dataset1_train_0/labels/6b52f8bd85ad.txt' mode='r' encoding='UTF-8'>
0 0.7328125 0.4890625 0.73125 0.490625 0.7296875 0.490625 0.7203125 0.5 0.7203125 0.5015625 0.71875 0.503125 0.71875 0.5140625 0.7203125 0.515625 0.7203125 0.51875 0.725 0.5234375 0.7265625 0.5234375 0.728125 0.525 0.7359375 0.525 0.7375 0.5265625 0.74375 0.5265625 0.7453125 0.528125 0.7515625 0.528125 0.753125 0.5296875 0.765625 0.5296875 0.765625 0.5203125 0.7640625 0.51875 0.7640625 0.5078125 0.7625 0.50625 0.7625 0.503125 0.7609375 0.5015625 0.7609375 0.496875 0.759375 0.4953125 0.759375 0.49375 0.75625 0.490625 0.753125 0.490625 0.7515625 0.4890625
0 0.7 0.6234375 0.696875 0.6265625 0.696875 0.6296875 0.6953125 0.63125 0.6953125 0.6328125 0.69375 0.634375 0.69375 0.6375 0.6921875 0.6390625 0.6921875 0.6484375 0.690625 0.65 0.690625 0.6578125 0.6890625 0.659375 0.6890625 0.6609375 0.69375 0.6609375 0.6953125 0.6625 0.7140625 0.6625 0.715625 0.6609375 0.7171875 0.

In [ ]:
def check_duplicate_labels(coords1, coords2):
    coords1 = [float(coord.replace('\n', '')) for coord in coords1]
    coords2 = [float(coord.replace('\n', '')) for coord in coords2]
    return all([x==y for x,y in zip(coords1, coords2)])

In [ ]:
for fold in folds:
  for train_mode in train_modes:
    yolo_annot_dir = f'{base_path}/{base_data_dir}/yolo_dataset1_{train_mode}_{fold}/labels'
    yolo_annot_files = os.listdir(yolo_annot_dir)
    for yolo_annot_file in yolo_annot_files:
        with open(f'{yolo_annot_dir}/{yolo_annot_file}', 'r') as f:
            full_annot_data = f.readlines()
        all_coords = [line.split(' ') for line in full_annot_data]
        for i in range(len(all_coords)):
            for j in range(i+1, len(all_coords)):
                if check_duplicate_labels(all_coords[i], all_coords[j]):
                    print(f'Found duplicate coords in file {yolo_annot_file}')

In [5]:
custom_config = """lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.3  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 0.7  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.9  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.1  # image mixup (probability)
copy_paste: 0.1  # segment copy-paste (probability)"""

In [11]:
custom_config_lines = custom_config.split('\n')
custom_dict = {custom_config_line.split()[0][:-1]: custom_config_line.split()[1] for custom_config_line in custom_config_lines }
custom_dict

{'lr0': '0.01',
 'lrf': '0.1',
 'momentum': '0.937',
 'weight_decay': '0.0005',
 'warmup_epochs': '3.0',
 'warmup_momentum': '0.8',
 'warmup_bias_lr': '0.1',
 'box': '0.05',
 'cls': '0.3',
 'cls_pw': '1.0',
 'obj': '0.7',
 'obj_pw': '1.0',
 'iou_t': '0.20',
 'anchor_t': '4.0',
 '': 'anchors:',
 'fl_gamma': '0.0',
 'hsv_h': '0.015',
 'hsv_s': '0.7',
 'hsv_v': '0.4',
 'degrees': '0.0',
 'translate': '0.1',
 'scale': '0.9',
 'shear': '0.0',
 'perspective': '0.0',
 'flipud': '0.0',
 'fliplr': '0.5',
 'mosaic': '1.0',
 'mixup': '0.1',
 'copy_paste': '0.1'}